In [1]:
!pip install -U ipywidgets

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 25.9 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.7
    Uninstalling widgetsnbextension-3.6.7:
      Successfully uninstalled widgetsnbextension-3.6.7
  Attempting uninstall: jupyterlab_widgets
    Found existing installation: jupyterlab_widgets 1.1.8
    Uninstalling jupyterlab_widgets-1.1.8:
      Successfully uninstalled jupyterlab_widgets-1.1.8
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.2
    Uninstalling ipywidgets-7.7.2:
      Successfully uninstalled ipywidgets-7.7.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [ipywidgets]3 [ipywidgets]


In [9]:
import gym
import time
from gym.envs.registration import register
import argparse
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.connectors.env_to_module import FlattenObservations

In [4]:
register(
            id='multigrid-collect-v0',
            entry_point='gym_multigrid.envs:CollectGame4HEnv10x10N2',
        )
env = gym.make('multigrid-collect-v0')

In [44]:
_ = env.reset()
nb_agents = len(env.agents)

for _ in range(1):
    #env.render(highlight=True)
    time.sleep(0.1)
    
    ac = [env.action_space.sample() for _ in range(nb_agents)]
    #ac = [0] * nb_agents
    
    obs, _, done, _ = env.step(ac)
    print("Num Agents = ", len(obs))
    print("Obs shape = ", obs[0].T.shape)
    
    if done:
        break

Num Agents =  3
Obs shape =  (6, 7, 7)


In [45]:
# Agent
import numpy as np
import argparse
import gym
from mappo.onpolicy.algorithms.r_mappo.r_mappo import R_MAPPO
from mappo.onpolicy.algorithms.r_mappo.algorithm.rMAPPOPolicy import R_MAPPOPolicy as policy
from mappo.onpolicy.config import get_config

In [46]:
''' rMAPPOPolicy wants a centralized observation space as well. In this case, what we could do is output the whole grid, but that might
cause some unnessessary strife here. It's probably easier in this case to just throw all of the observations into one array.'''

obs_shape = obs[0].T.shape
obs_shape = gym.spaces.Box(0, 32*32, obs_shape)

action_space = gym.spaces.Discrete(3) # One hot encoded action space

cent_obs_space = np.array([4, 6, 7, 7])
cent_obs_space = gym.spaces.Box(0, 32*32, cent_obs_space)

# Gets the default arguments for the policy in question
parser = get_config()                        # build the ArgumentParser
args   = parser.parse_args([])               # empty list ⇒ use every default
#print(vars(args))

p = policy(args, obs_space = obs_shape, cent_obs_space = cent_obs_space, act_space = action_space)
agent = R_MAPPO(args, policy)